# Dino Fine-Tuning - Simple

In [ ]:
!pip install -q transformers>=4.40 accelerate peft

In [ ]:
!rm -rf Dino-Training
!git clone https://github.com/RekitRex21/Dino-Training.git
%cd Dino-Training

In [ ]:
import json, os
data = []
for f in os.listdir('.'):
  if f.endswith('.json') or f.endswith('.jsonl'):
    with open(f) as fp:
      if f.endswith('.jsonl'): data.extend([json.loads(l) for l in fp])
      else: data.extend(json.load(fp))
print(f"Loaded {len(data)} examples")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
m = 'microsoft/Phi-3-mini-4k-instruct'
tok = AutoTokenizer.from_pretrained(m, trust_remote_code=True)
tok.pad_token = tok.eos_token
mod = AutoModelForCausalLM.from_pretrained(m, device_map='cpu', trust_remote_code=True)
print('Model loaded (CPU mode)')

In [ ]:
texts = [f"### Instruction\n{d.get('instruction','')}\n\n### Input\n{d.get('input','')}\n\n### Response\n{d['output']}" for d in data]

In [ ]:
from datasets import Dataset
ds = Dataset.from_dict({'text': texts[:10]}) # Use 10 examples for quick test
def tok_fn(ex): return tok(ex['text'], truncation=True, max_length=256)
tokenized = ds.map(tok_fn, batched=True)

In [ ]:
from peft import LoraConfig, get_peft_model
lc = LoraConfig(r=4, lora_alpha=8, target_modules=['q_proj'], lora_dropout=0.1, bias='none', task_type='CAUSAL_LM')
mod = get_peft_model(mod, lc)
mod.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir='./out', num_train_epochs=1, per_device_train_batch_size=1, logging_steps=1)
trainer = Trainer(model=mod, args=args, train_dataset=tokenized)
trainer.train()
print('Training done!')

In [ ]:
print('For full training: increase examples, epochs, use GPU colab')